<a href="https://colab.research.google.com/github/ShubhamMutreja/Sonar-Prediction-model/blob/master/Sonar_Model_pred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Importing required packages.
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/undocumented/connectionist-bench/sonar/sonar.all-data'

In [0]:
def read_dataset():
  dt = pd.read_csv(url)
  X = dt[dt.columns[0:60]].values
  Y = dt[dt.columns[60]]
  return X,Y

In [0]:
def encoder(X,Y):
  ecod = LabelEncoder()
  ecod.fit(Y)
  
  Y = ecod.transform(Y)
  Y = OHE(Y)
  return X,Y

In [0]:
def OHE(Y):
  labels = len(Y)
  unique = len(np.unique(Y))
  OHE = np.zeros((labels , unique))
  OHE[np.arange(labels) , Y] = 1
  return OHE

In [0]:
def tensorflow_multilayer_precerptron(X , W , B):
  # Hidden layer with sigmoid activated
  layer_1 = tf.add(tf.matmul(x,weights['h1']), biases['b1'])
  layer_1 = tf.nn.sigmoid(layer_1)

  # Hidden layer with sigmoid activated
  layer_2 = tf.add(tf.matmul(layer_1,weights['h2']), biases['b2'])
  layer_2 = tf.nn.sigmoid(layer_2)
      
  # Hidden layer with sigmoid activated
  layer_3 = tf.add(tf.matmul(layer_2,weights['h3']), biases['b3'])
  layer_3 = tf.nn.sigmoid(layer_3)
    
  # Hidden layer with RELU activated
  layer_4 = tf.add(tf.matmul(layer_3,weights['h4']), biases['b4'])
  layer_4 = tf.nn.relu(layer_4)
  
  # Output layer with linear activation
  out_layer = tf.matmul(layer_4, weights['out']+biases['out'])
  return out_layer
    

In [6]:
# Read the dataset
X, Y = read_dataset()
X, Y = encoder(X,Y)
# Shuffle the dataset to mix up the rows
X, Y = shuffle(X, Y, random_state = 1)

# Convert the dataset into train and test part
train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size=0.2, random_state=415)

#Inspect the shape of the training and testing
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(165, 60)
(165, 2)
(42, 60)
(42, 2)


In [7]:
# Define the important parametrs and variables to work with the tensors
learning_rate = 0.02
training_epochs = 1000
cost_history = np.empty(shape = [1], dtype=float)
n_dim = X.shape[1]
print("n_dim",n_dim)

n_class = 2
model_path = "./model"

# Define the number of hidden layers and number of neurons for each layer
n_hidden_1 = 60
n_hidden_2 = 70
n_hidden_3 = 70
n_hidden_4 = 60

x = tf.placeholder(tf.float32,[None,n_dim]) # None means any value
W = tf.Variable(tf.zeros([n_dim,n_class]))
b = tf.Variable(tf.zeros([n_class]))
y_true = tf.placeholder(tf.float32,[None,n_class])

weights = {
    'h1': tf.Variable(tf.truncated_normal([n_dim,n_hidden_1])),
    'h2': tf.Variable(tf.truncated_normal([n_hidden_1,n_hidden_2])),
    'h3': tf.Variable(tf.truncated_normal([n_hidden_2,n_hidden_3])),
    'h4': tf.Variable(tf.truncated_normal([n_hidden_3,n_hidden_4])),
    'out': tf.Variable(tf.truncated_normal([n_hidden_4,n_class]))
}

biases = {
    'b1': tf.Variable(tf.truncated_normal([n_hidden_1])),
    'b2': tf.Variable(tf.truncated_normal([n_hidden_2])),
    'b3': tf.Variable(tf.truncated_normal([n_hidden_3])),
    'b4': tf.Variable(tf.truncated_normal([n_hidden_4])),
    'out': tf.Variable(tf.truncated_normal([n_class]))
}

# Initialize all the variables
init = tf.global_variables_initializer()

#  Saver object to save our model
saver = tf.train.Saver()

# Call your model defined
y = tensorflow_multilayer_precerptron(x, weights, biases)

# Define the cost function and optimizer
cost_function = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y, labels=y_true))
training_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost_function)

n_dim 60
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [8]:
sess = tf.Session()
sess.run(init)

# Calculate the cost and accuracy for each epoch

mse_history = []
accuracy_history = []
print("**")
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)


for epoch in range(training_epochs):
    sess.run(training_step, feed_dict={x:train_x, y_true:train_y})
    cost = sess.run(cost_function, feed_dict={x:train_x, y_true:train_y})
    cost_history = np.append(cost_history, cost)
    correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(y_true,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    
    pred_y = sess.run(y,feed_dict={x:test_x})
    mse = tf.reduce_mean(tf.square(pred_y - test_y))
    mse_ = sess.run(mse)
    mse_history.append(mse_)
    accuracy = sess.run(accuracy,feed_dict={x: train_x, y_true:train_y})
    accuracy_history.append(accuracy)
    
    print('epoch: ',epoch,' - ','cost:',cost,' -MSE:',mse_," -Train Accuracy: ",accuracy)
save_path = saver.save(sess, model_path)

**
(165, 60)
(165, 2)
(42, 60)
(42, 2)
epoch:  0  -  cost: 37.59929  -MSE: 1524.6556002579089  -Train Accuracy:  0.45454547
epoch:  1  -  cost: 35.584095  -MSE: 1790.3216188455674  -Train Accuracy:  0.54545456
epoch:  2  -  cost: 8.406357  -MSE: 226.81887898146817  -Train Accuracy:  0.45454547
epoch:  3  -  cost: 28.985447  -MSE: 1194.682778949354  -Train Accuracy:  0.54545456
epoch:  4  -  cost: 1.2651925  -MSE: 111.4162897877869  -Train Accuracy:  0.6060606
epoch:  5  -  cost: 1.3380088  -MSE: 115.95546113740035  -Train Accuracy:  0.54545456
epoch:  6  -  cost: 9.568236  -MSE: 195.49319468554836  -Train Accuracy:  0.45454547
epoch:  7  -  cost: 14.4904785  -MSE: 375.78982842955446  -Train Accuracy:  0.54545456
epoch:  8  -  cost: 2.3622515  -MSE: 82.29780687435427  -Train Accuracy:  0.4969697
epoch:  9  -  cost: 11.766576  -MSE: 264.51789819984765  -Train Accuracy:  0.54545456
epoch:  10  -  cost: 1.4833382  -MSE: 60.95569512409882  -Train Accuracy:  0.5272727
epoch:  11  -  cost: 6.